In [2]:
api_key = 

In [3]:
from googleapiclient.discovery import build
import pandas as pd
from IPython.display import JSON

In [4]:
# The channels which are to be analysed need to be added here

channel_ids = [
    'UC0intLFzLaudFG-xAvUEO-A', # https://www.youtube.com/c/NotJustBikes
    'UCGc8ZVCsrR3dAuhvUbkbToQ', # https://www.youtube.com/c/CityBeautiful
    'UCfgtNfWCtsLKutY-BHzIb9Q', # https://www.youtube.com/c/CityNerd
    'UC-LM91jkqJdWFvm9B5G-w7Q', # https://www.youtube.com/c/AlanFisher1337
    'UCK1XzxcKXB_v_dcw_tflC1A' # https://www.youtube.com/c/OBFYT
    ]

In [5]:
api_service_name = "youtube"
api_version = "v3"
# Get credentials and create an API client
youtube = build(
    api_service_name, api_version, developerKey=api_key)

In [9]:
def get_channel_stats (youtube, channel_ids):
    all_data = []
    
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=','.join(channel_ids)
    )
    response = request.execute()
    
    #  loop through all the analysed videos' JSON files
    for item in response['items']:
        data = {
            'channelName': item['snippet']['title'],
            'subscribers': item['statistics']['subscriberCount'],
            'views': item['statistics']['viewCount'],
            'totalVideos': item['statistics']['videoCount'],
            'playlistId': item['contentDetails']['relatedPlaylists']['uploads']
        }
        
        all_data.append(data)
        
    return(pd.DataFrame(all_data))

In [10]:
channel_stats = get_channel_stats(youtube, channel_ids)

In [11]:
channel_stats

,channelName,subscribers,views,totalVideos,playlistId
0,Alan Fisher,123000,9344638,57,UU-LM91jkqJdWFvm9B5G-w7Q
1,Not Just Bikes,714000,69737467,72,UU0intLFzLaudFG-xAvUEO-A
2,OBF,388000,49083443,68,UUK1XzxcKXB_v_dcw_tflC1A
3,City Beautiful,529000,55546512,117,UUGc8ZVCsrR3dAuhvUbkbToQ
4,CityNerd,60900,2789923,53,UUfgtNfWCtsLKutY-BHzIb9Q


In [28]:
# playlist_ids = list(channel_stats['playlistId'])
playlist_id = 'UU-LM91jkqJdWFvm9B5G-w7Q'

def get_video_ids(youtube, playlist_id):
    
    video_ids = []
    request = youtube.playlistItems().list(
        part="snippet, contentDetails",
        playlistId=playlist_id,
        maxResults = 50
    )
    response = request.execute()

    for item in response['items']:
        video_ids.append(item['contentDetails']['videoId'])

    next_page_token = response.get('nextPageToken')

    while next_page_token is not None:
        request = youtube.playlistItems().list(
            part="contentDetails",
            playlistId=playlist_id,
            maxResults = 50,
            pageToken = next_page_token
        )
        response = request.execute()

        for item in response['items']:
            video_ids.append(item['contentDetails']['videoId'])

        next_page_token = response.get('nextPageToken')
        
    return video_ids

In [29]:
video_ids = get_video_ids(youtube, playlist_id)

In [30]:
len(video_ids)

57

In [38]:
def get_video_details(youtube, video_ids):
    
    all_video_info = []

    for i in range (0, len(video_ids), 50):
        request = youtube.videos().list(
            part="snippet, contentDetails, statistics",
            id=','.join(video_ids[i:i+50])
        )
        response = request.execute()

        for video in response['items']:
            stats_to_keep = {
                'snippet': ['channelTitle', 'title', 'description', 'tags', 'publishedAt'],
                'statistics': ['viewCount', 'likeCount', 'favouriteCount', 'commentCount'],
                'contentDetails' : ['duration', 'definition', 'caption']
            }

            video_info = {}
            video_info['video_id'] = video['id']

            for k in stats_to_keep.keys():
                for v in stats_to_keep[k]:
                    try:
                        video_info[v] = video[k][v]
                    except:
                        video_info[v] = None
            all_video_info.append(video_info)

    return pd.DataFrame(all_video_info)

def get_comments_in_videos(youtube, video_ids):
    all_comments = []
    
    for video_id in video_ids:
        try:
            request = youtube.commentThreads().list(
                part = 'snippet,replies',
                videoId = video_id
            )
            response = request.execute()

            comments_in_video = [comment['snippet']['topLevelComment']['snippet']['textOriginal'] for comment in response['items'][0:10]]
            comments_in_video_info = {'video_id': video_id, 'comments': comments_in_video}

            all_comments.append(comments_in_video_info)
        except:
            print('Error getting comments on video ' + video_id)
        
    return pd.DataFrame(all_comments)

In [39]:
video_df = get_video_details(youtube, video_ids)
video_df 

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption
0,4QlZx1sjg4Y,Alan Fisher,"Grid Cities are Fine, and OBF is a Copycat",The difference between you and me is that I do...,"[armchair urbanist, OBF, city design, Urban pl...",2022-07-02T14:00:02Z,206381,16787,None,1742,PT13M36S,hd,false
1,Rt8W9iGLRA4,Alan Fisher,The Worst Transit Project in America and the F...,"yes, I know that I spelled Broad Street wrong....","[armchair urbanist, septa, norristown high spe...",2022-06-25T14:00:14Z,132494,7808,None,894,PT9M9S,hd,false
2,zQ6EIxlbUOk,Alan Fisher,Are Monorails good? What Housing should we bui...,"Its like half in the bag, except with Trains.\...","[armchair urbanist, donoteat01, Well there's y...",2022-06-08T14:00:32Z,39013,1918,None,376,PT1H34M45S,hd,false
3,rcjr4jbGuJg,Alan Fisher,California High Speed Rail has not Failed and ...,Your ability to put stock videos over a script...,"[armchair urbanist, CAHSR, trains, urban plann...",2022-05-31T14:00:21Z,692773,44484,None,5772,PT17M52S,hd,false
4,JbqNUqdZlwM,Alan Fisher,Why Philadelphia's Streets are better than oth...,when the road be too girthy\n\nThe Bike:\nhttp...,"[armchair urbanist, urban planning, not just b...",2022-05-12T14:00:07Z,398403,14196,None,1317,PT8M8S,hd,false
5,AOndVouUSRA,Alan Fisher,"Real Solar Punk is Smart Land Use, Not Gimmick...","Farms good, suburbs bad.\n\nGo follow Miles!\n...","[urban planning, not just bikes, transit, trai...",2022-05-03T14:00:03Z,174629,13193,None,1374,PT5M58S,hd,false
6,0XrBoasOeiY,Alan Fisher,Ranking the 12 Largest American Cities on Urba...,Which American city is the worst? Find out and...,"[armchair urbanist, city planning, tier list, ...",2022-03-08T14:00:02Z,141192,6623,None,1489,PT38M20S,hd,false
7,JGG5WRBPeFk,Alan Fisher,Parking Garages Represent Lazy and Terrible Pl...,"Car building bad, human building good.\n\nThe ...","[car parks, urban planning, not just bikes, tr...",2022-02-22T13:30:04Z,177505,10095,None,1387,PT6M38S,hd,false
8,0qIsUyidFjs,Alan Fisher,You Deserve Better Transit #shorts,The Armchair Urbanist Series: \nhttps://www.yo...,"[armchair urbanist, urban planning, trains, am...",2022-01-27T14:00:00Z,58205,6831,None,555,PT39S,hd,false
9,nJNvpG5gktM,Alan Fisher,Parallel Systems Autonomous Pod Train is an Ob...,SpaceX Engineers *derogatory*\n\nThe Armchair ...,"[armchair urbanist, freight rail, tesla, elon ...",2022-01-21T14:00:21Z,667855,24732,None,2659,PT9M39S,hd,false


In [42]:
comments = get_comments_in_videos(youtube, video_ids)
comments['comments'][0]

["Hi Alan, first of all thank you for this video. I’ll do my best to explain everything in this comment to clear some of the stuff up that you mention.\n\nThe first thing is that you disagree with the video, obviously you’re quite knowledgeable about this topic and I think you presented all the points well. I’ll definitely take the criticisms to heart. I haven’t really thought much about including the sources but it makes a lot of sense to do. I’ll go back and add those. Hopefully they will bring some clarity to my points in the video. I will say though that it was an opinion piece but I could probably have done a better job making that more clear.\n\nThe second thing is plagiarism. The section of my video you show where I used Vox’s words instead of my own is 100% plagiarism and I’ve cut that part out of the video. It should have never been in the video in the first place, I don't know what I was thinking and I’m truly ashamed of myself for this.\nI’ve always looked to Vox for inspira